```{contents}
```
## RunnableSequence


**RunnableSequence** represents a **linear, ordered pipeline** of runnables where the **output of one step becomes the input of the next**.
In LangChain’s LCEL (LangChain Expression Language), a sequence is most commonly written using the pipe operator `|`.

Provided by LangChain.

```
Input
  → Step 1
  → Step 2
  → Step 3
  → Output
```

---

### Why RunnableSequence Is Needed

Most LLM workflows are **sequential by nature**:

* Normalize input
* Build prompt
* Call LLM
* Parse output

RunnableSequence provides:

* Deterministic execution order
* Clear data flow
* Composability
* Readable pipelines

---

### Conceptual Flow

```
User Input
   ↓
Runnable A (transform)
   ↓
Runnable B (format)
   ↓
Runnable C (LLM)
   ↓
Final Output
```

Each step **depends on the previous one**.

---

### Minimal RunnableSequence Example



In [1]:
from langchain_core.runnables import RunnableLambda

sequence = (
    RunnableLambda(lambda x: x.strip())
    | RunnableLambda(lambda x: x.upper())
)

sequence.invoke("  runnable sequence  ")

'RUNNABLE SEQUENCE'



**Output**

```
RUNNABLE SEQUENCE
```

Here:

* Step 1 trims whitespace
* Step 2 uppercases text

---

### RunnableSequence with Prompt + LLM



In [2]:
from langchain_classic.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template(
    "Explain the following concept briefly:\n{concept}"
)

chain = (
    RunnableLambda(lambda x: {"concept": x})
    | prompt
    | llm
)
chain.invoke("RunnableSequence")

AIMessage(content='A RunnableSequence is a sequence of runnable tasks that can be executed concurrently in a multi-threaded environment. Each task is independent of the others and can be run in parallel, allowing for efficient utilization of system resources and increased performance. This concept is commonly used in programming to execute multiple tasks simultaneously.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 16, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cpyyd4DHddr150379ok2QK9tUX6sp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b4be9-9875-7502-9221-6093c2a25660-0', usage_metadata={'



```python

```

Execution order:

1. Wrap input into dict
2. Render prompt
3. Call LLM

---

### RunnableSequence in RAG (Sequential Dependency)



In [5]:
# Create a simple retriever for demonstration
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_classic.schema import Document

# Sample documents
docs = [
    Document(page_content="LangChain is a framework for developing applications powered by language models."),
    Document(page_content="RunnableSequence executes steps in order, where each step depends on the previous one."),
    Document(page_content="Vector stores enable semantic search over documents using embeddings.")
]

# Create vector store and retriever
vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [7]:
chain = (
    retriever                 # input → documents
    | RunnableLambda(lambda d: "\n".join(x.page_content for x in d))
    | ChatPromptTemplate.from_template(
        "Answer using the context:\n{context}"
    )
    | llm
)
chain.invoke("What is LangChain?")

AIMessage(content='LangChain provides a platform for developing applications that utilize language models, while RunnableSequence allows for the sequential execution of steps in an application where each step builds upon the outcome of the previous one. Vector stores, on the other hand, enable semantic search capabilities over documents through the use of embeddings.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 51, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cpz0QwgQ3HAZJgoIrDfdDE9BxVZNY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b4beb-493c-7eb2-b418-16d2b0a60ea9-0', usage_metadata={'in



Why this must be a sequence:

* Retriever output is **required** before prompt
* Prompt output is **required** before LLM

---

### Explicit RunnableSequence (Alternative Form)



In [8]:
from langchain_core.runnables import RunnableSequence

sequence = RunnableSequence(
    RunnableLambda(lambda x: x + 1),
    RunnableLambda(lambda x: x * 2),
    RunnableLambda(lambda x: x - 3),
)

sequence.invoke(5)

9



**Output**

```
9
```

Calculation:

```
(5 + 1) * 2 - 3 = 9
```

---

### RunnableSequence vs RunnableParallel

| Aspect     | RunnableSequence | RunnableParallel |
| ---------- | ---------------- | ---------------- |
| Execution  | Ordered          | Concurrent       |
| Dependency | Yes              | No               |
| Output     | Single value     | Dict             |
| Use case   | Pipelines        | Fan-out          |

---

### RunnableSequence vs RunnableLambda

| Component        | Role                  |
| ---------------- | --------------------- |
| RunnableLambda   | Single transformation |
| RunnableSequence | Ordered composition   |

RunnableSequence is built **from** smaller runnables.

---

### Common Use Cases

* Input normalization → LLM
* Retrieval → formatting → generation
* Output parsing
* Guardrails before model calls
* Deterministic business logic

---

### Common Mistakes

* Using sequence when parallelism is needed
* Forgetting output type compatibility between steps
* Mixing dict and string shapes unintentionally

Rule:

> Output of step *n* must match input of step *n+1*

---

### Mental Model

RunnableSequence is the **“pipeline conveyor belt”** of LCEL.

```
One step finishes → next step starts
```

---

### Best Practices

* Keep steps small and explicit
* Use RunnableLambda for glue logic
* Use RunnableParallel only when no dependency exists
* Combine Sequence + Parallel for complex graphs

---

### Key Takeaways

* RunnableSequence defines **ordered execution**
* Implemented with `|`
* Core building block of LCEL
* Essential for deterministic LLM pipelines